In [2]:
# https://spark.apache.org/examples.html --- Reference Website


In [3]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [6]:
# Read the DataSet
training = spark.read.csv('Emp_Salary.csv',header = True,inferSchema= True)

In [7]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [9]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [10]:
# now we will try to find out the expected salary on the basis of Age and Experince
training.columns

['Name', 'age', 'Experience', 'Salary']

In [12]:
# will try to create group of independent variable 
[Age,Experience]---> new feature ----> independent Features



In [17]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols = ["age","Experience"],outputCol="Independent Features")

In [19]:
output = featureassembler.transform(training)

In [35]:
#?featureassembler.transform
output.printSchema

<bound method DataFrame.printSchema of DataFrame[Name: string, age: int, Experience: int, Salary: int, Independent Features: vector]>

In [20]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [21]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [23]:
finalized_data = output.select("Independent Features","Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [36]:
from pyspark.ml.regression import LinearRegression
# train Emp_Salary split
train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = 'Independent Features',labelCol = 'Salary')
regressor = regressor.fit(train_data)


In [37]:
test_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
+--------------------+------+



In [38]:
### coefficients 
regressor.coefficients

DenseVector([28.4757, 1271.3568])

In [39]:
regressor.intercept

14299.832495812996

In [40]:
### prediction
pred_results = regressor.evaluate(test_data)

In [41]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|         [31.0,10.0]| 30000|27896.147403685147|
+--------------------+------+------------------+



In [42]:
pred_results.meanAbsoluteError




2103.852596314853

In [43]:
pred_results.meanSquaredError


4426195.747020748